In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.regularizers import l2


# path to file
dataFile = './cifar10/test_batch'
metaFile = './cifar10/batches.meta'

def plottingModel():
    # Plot accuracy
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Model Accuracy')
    plt.show()

    # Plot loss
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Model Loss')
    plt.show()

# loading CIFAR-10 data & metadata
def loadData(fileName, normalize=True):
    with open(fileName, 'rb') as f:
        dataDict = pickle.load(f, encoding='bytes')
        if b'data' in dataDict:
            data = dataDict[b'data']
            labels = dataDict[b'labels']
            # reshaping & transposing to correct dimensions
            images = data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
            # normalizing pixel values between range [0, 1]
            if normalize:
                images = images / 255.0
            return images, np.array(labels)
        if b'label_names' in dataDict:
            return [name.decode('utf-8') for name in dataDict[b'label_names']]

# load data & metadata
images, labels = loadData(dataFile)
labelNames = loadData(metaFile)
imageShape = images[0].shape
print(f"Data shape: {images.shape}, Labels shape: {labels.shape}")
print(f"Pixel value range: Min={images.min()}, Max={images.max()}")
print(f"Label Names: {labelNames}")
print(f'SHAPE IMAGE: {imageShape}')
# print(f'Image example:{images[0]}')


def displayImages(images, labels, labelNames, numImages=5):
    """displayes first few images w/ labels
    """
    for i in range(numImages):
        plt.imshow((images[i] * 255).astype('uint8'))  # Scale back to [0, 255] for display
        plt.title(labelNames[labels[i]])
        plt.xticks([])
        plt.yticks([])
        plt.show()

def displayRandomGrid(images, labels, labelNames, rows=5, cols=5):
    """
    displays a random grid of imgs
    """
    fig = plt.figure(figsize=(10, 10))
    randomIdx = np.random.randint(0, len(images), rows * cols)
    for i, idx in enumerate(randomIdx):
        fig.add_subplot(rows, cols, i + 1)
        plt.imshow((images[idx] * 255).astype('uint8'))  # Scale back to [0, 255] for display
        plt.title(labelNames[labels[idx]])
        plt.xticks([])
        plt.yticks([])
    plt.show()

# # displays:
# displayImages(images, labels, labelNames)
# displayRandomGrid(images, labels, labelNames)

uniqueLabels= int(len(np.unique(labels)))
labelsCategorical = to_categorical(labels, num_classes=uniqueLabels)
print("One hot encoded categorical label example:",labelsCategorical[0])


Data shape: (10000, 32, 32, 3), Labels shape: (10000,)
Pixel value range: Min=0.0, Max=1.0
Label Names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
SHAPE IMAGE: (32, 32, 3)
One hot encoded categorical label example: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [7]:
model = Sequential()
# First convolutional block
model.add(Conv2D(128, kernel_size=3, activation="relu", input_shape=imageShape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Second convolutional block
model.add(Conv2D(256, kernel_size=3, activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Third convolutional block
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

# Flatten and dense layers
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


In [3]:
model=Sequential()
model.add(Conv2D(128, kernel_size=3, activation="relu", input_shape=imageShape))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) #connects convolution and dense layer
model.add(Dense(10, activation='softmax')) #10 probs added to probabilities

#compile model using accuracy
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#splitting data
X_train, X_test, y_train, y_test = train_test_split(images, labelsCategorical, random_state=0, train_size=.7, shuffle=True)
#introducing early stopping for overfitting
earlyStopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
#training model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[earlyStopping])
test_loss, test_acc = model.evaluate(images, labelsCategorical, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")
print("MAX Validation Accuracy", max(list(history.history['val_accuracy'])))


Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.3059 - loss: 2.4945 - val_accuracy: 0.2020 - val_loss: 2.3990
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 14s 66ms/step - accuracy: 0.4850 - loss: 1.6255 - val_accuracy: 0.2407 - val_loss: 2.3030
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 13s 57ms/step - accuracy: 0.5515 - loss: 1.3650 - val_accuracy: 0.3810 - val_loss: 1.8333
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 53ms/step - accuracy: 0.5878 - loss: 1.2629 - val_accuracy: 0.4973 - val_loss: 1.4921
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.6213 - loss: 1.1412 - val_accuracy: 0.5087 - val_loss: 1.5018
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.6286 - loss: 1.0838 - val_accuracy: 0.4640 - val_loss: 1.7467
313/313 - 4s - 14ms/step - accuracy: 0.5460 - loss: 1.3290
Test Accuracy: 0.5460
MAX Validation Accuracy 0.5086666941642761


max pool 3,3

In [4]:
model = Sequential()
model.add(Conv2D(128, kernel_size=3, activation="relu", input_shape=imageShape))
# model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
#compile model using accuracy
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#splitting data
X_train, X_test, y_train, y_test = train_test_split(images, labelsCategorical, random_state=0, train_size=.7, shuffle=True)
#introducing early stopping for overfitting
earlyStopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
#training model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[earlyStopping])
test_loss, test_acc = model.evaluate(images, labelsCategorical, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")
print("MAX Validation Accuracy", max(list(history.history['val_accuracy'])))


Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.1744 - loss: 2.5005 - val_accuracy: 0.3663 - val_loss: 1.8202
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 56ms/step - accuracy: 0.3550 - loss: 1.8388 - val_accuracy: 0.4410 - val_loss: 1.7020
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 57ms/step - accuracy: 0.4225 - loss: 1.7022 - val_accuracy: 0.4627 - val_loss: 1.5801
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 56ms/step - accuracy: 0.4489 - loss: 1.6315 - val_accuracy: 0.4507 - val_loss: 1.6200
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 57ms/step - accuracy: 0.4840 - loss: 1.5574 - val_accuracy: 0.4927 - val_loss: 1.5495
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 13s 58ms/step - accuracy: 0.4982 - loss: 1.5413 - val_accuracy: 0.4617 - val_loss: 1.6075
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 12s 57ms/step - accuracy: 0.5280 - loss: 1.4693 - val_accuracy: 0.4997 - val_loss: 1.5321
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 13s 57ms/step - accuracy: 0.5490 - loss: 1.4309 - 

No max pool at all

In [5]:
model = Sequential()
model.add(Conv2D(128, kernel_size=3, activation="relu", input_shape=imageShape))
# model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
#compile model using accuracy
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#splitting data
X_train, X_test, y_train, y_test = train_test_split(images, labelsCategorical, random_state=0, train_size=.7, shuffle=True)
#introducing early stopping for overfitting
earlyStopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
#training model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[earlyStopping])
test_loss, test_acc = model.evaluate(images, labelsCategorical, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")
print("MAX Validation Accuracy", max(list(history.history['val_accuracy'])))

Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 17s 73ms/step - accuracy: 0.2066 - loss: 2.4245 - val_accuracy: 0.3807 - val_loss: 1.8924
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 15s 70ms/step - accuracy: 0.3463 - loss: 1.9254 - val_accuracy: 0.3793 - val_loss: 1.8452
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 15s 70ms/step - accuracy: 0.3963 - loss: 1.8165 - val_accuracy: 0.3977 - val_loss: 1.7684
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 17s 79ms/step - accuracy: 0.4092 - loss: 1.7838 - val_accuracy: 0.4253 - val_loss: 1.7240
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 18s 80ms/step - accuracy: 0.4287 - loss: 1.7381 - val_accuracy: 0.4557 - val_loss: 1.7045
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 18s 83ms/step - accuracy: 0.4535 - loss: 1.6986 - val_accuracy: 0.4403 - val_loss: 1.7053
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 18s 80ms/step - accuracy: 0.4671 - loss: 1.6585 - val_accuracy: 0.4330 - val_loss: 1.6965
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 18s 80ms/step - accuracy: 0.4668 - loss: 1.6512 - 

higher early stopping and no

In [6]:
model = Sequential()
model.add(Conv2D(128, kernel_size=3, activation="relu", input_shape=imageShape))
# model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Flatten())
# model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
#compile model using accuracy
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#splitting data
X_train, X_test, y_train, y_test = train_test_split(images, labelsCategorical, random_state=0, train_size=.7, shuffle=True)
#introducing early stopping for overfitting
earlyStopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
#training model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[earlyStopping])
test_loss, test_acc = model.evaluate(images, labelsCategorical, verbose=2)
print(f"Test Accuracy: {test_acc:.4f}")
print("MAX Validation Accuracy", max(list(history.history['val_accuracy'])))

Epoch 1/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 17s 72ms/step - accuracy: 0.2482 - loss: 2.0242 - val_accuracy: 0.4243 - val_loss: 1.6284
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 16s 72ms/step - accuracy: 0.4807 - loss: 1.4614 - val_accuracy: 0.4683 - val_loss: 1.4977
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 15s 71ms/step - accuracy: 0.5577 - loss: 1.2340 - val_accuracy: 0.5033 - val_loss: 1.4618
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 16s 72ms/step - accuracy: 0.6447 - loss: 1.0311 - val_accuracy: 0.5000 - val_loss: 1.4725
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 16s 72ms/step - accuracy: 0.6918 - loss: 0.8751 - val_accuracy: 0.4970 - val_loss: 1.5703
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 16s 74ms/step - accuracy: 0.7535 - loss: 0.7203 - val_accuracy: 0.4910 - val_loss: 1.6072
313/313 - 6s - 20ms/step - accuracy: 0.6083 - loss: 1.1399
Test Accuracy: 0.6083
MAX Validation Accuracy 0.503333330154419
